In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Load datasets
transaction_data = pd.read_excel("Datasets/transaction_data.xlsx")
product_data = pd.read_excel("Datasets/product_data.xlsx")

tran = transaction_data.copy()
prod = product_data.copy()

In [2]:
# dropna drops missing values
tran = tran.dropna(axis=0)
prod = prod.dropna(axis=0)
tran

,product_id,purchased_quantity,unit_product_price,transaction_id,customer_id,transaction_date
0,1,10,1,1,3,2012-09-15
1,1,2,1,2,2,2012-09-15
2,1,1,1,3,1,2012-09-15
3,2,1,2,4,2,2012-09-15
4,2,2,2,1,3,2012-09-15
5,2,7,2,3,2,2012-10-15
6,2,9,2,5,2,2012-10-15
7,3,3,3,6,1,2012-10-15
8,3,6,3,7,1,2012-10-15
9,3,6,3,6,1,2012-10-15


In [3]:
# Group the products by total purchased quantity
df1 = tran.groupby(['product_id'])['purchased_quantity'].sum().sort_values(ascending=False).reset_index()
df1

,product_id,purchased_quantity
0,2,19
1,3,15
2,1,13


In [4]:
# Find the most purchased product
pr = df1.product_id.iloc[0]

# Find the color of the most purchased product
cr = prod.loc[prod.product_id==pr, 'color'].unique()
cr

array(['b'], dtype=object)

In [5]:
# Convert the transaction date to datetime
tran['transaction_date'] = pd.to_datetime(tran['transaction_date'], format='%Y-%m-%d')
tran

,product_id,purchased_quantity,unit_product_price,transaction_id,customer_id,transaction_date
0,1,10,1,1,3,2012-09-15
1,1,2,1,2,2,2012-09-15
2,1,1,1,3,1,2012-09-15
3,2,1,2,4,2,2012-09-15
4,2,2,2,1,3,2012-09-15
5,2,7,2,3,2,2012-10-15
6,2,9,2,5,2,2012-10-15
7,3,3,3,6,1,2012-10-15
8,3,6,3,7,1,2012-10-15
9,3,6,3,6,1,2012-10-15


In [6]:
# Filter the data
df2 = tran[tran['transaction_date'] < '2012-10-01'].copy()
df2

,product_id,purchased_quantity,unit_product_price,transaction_id,customer_id,transaction_date
0,1,10,1,1,3,2012-09-15
1,1,2,1,2,2,2012-09-15
2,1,1,1,3,1,2012-09-15
3,2,1,2,4,2,2012-09-15
4,2,2,2,1,3,2012-09-15


In [7]:
# Create new features
df2['money_spent'] = df2['purchased_quantity'] * df2['unit_product_price']
df2

,product_id,purchased_quantity,unit_product_price,transaction_id,customer_id,transaction_date,money_spent
0,1,10,1,1,3,2012-09-15,10
1,1,2,1,2,2,2012-09-15,2
2,1,1,1,3,1,2012-09-15,1
3,2,1,2,4,2,2012-09-15,2
4,2,2,2,1,3,2012-09-15,4


In [8]:
# Find the product id having color 'b'
a_pr = prod.loc[prod.color=='b', 'product_id'].unique()
a_pr

array([2])

In [9]:
# If customer bought a product 'a'
df2['product'] = df2['product_id'].apply(lambda k: 1 if k==a_pr else 0)
df2

,product_id,purchased_quantity,unit_product_price,transaction_id,customer_id,transaction_date,money_spent,product
0,1,10,1,1,3,2012-09-15,10,0
1,1,2,1,2,2,2012-09-15,2,0
2,1,1,1,3,1,2012-09-15,1,0
3,2,1,2,4,2,2012-09-15,2,1
4,2,2,2,1,3,2012-09-15,4,1


In [10]:
# Aggregate data by customer
df3 = df2.groupby('customer_id').agg({
'transaction_id': lambda n: len(n),
'money_spent': lambda m: m.sum(),
'product': lambda k: k.max()})

# Change the name of columns
df3.columns = ['number_transactions','total_money_spent','bought_product_b']

# Add response feature
df3['response'] = tran.groupby('customer_id').agg({'transaction_date': lambda x: x.max()})
df3['response'] = (df3['response'] >= '2012-10-01').astype(int)
df3

,number_transactions,total_money_spent,bought_product_b,response
customer_id,,,,
1,1,1,0,1
2,2,4,1,1
3,2,14,1,0


In [11]:
df3.to_csv("Datasets/feature_data.csv")

In [12]:
X = df3.copy()
y = X.pop('response')

X = StandardScaler().fit_transform(X)

In [13]:
# Split data into training and validation data, for both features and target
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.7, test_size=0.3,random_state=0)

In [ ]:
# SVC model
clf = svm.SVC(C=0.8, kernel='rbf', gamma=20)
clf.fit(X_train, y_train)

score = clf.score(X_train, y_train)
print("Score: ", score)

y_pred = clf.predict(X_valid)

cm = confusion_matrix(y_valid, y_pred)
sns.heatmap(cm, annot=True)

cr = classification_report(y_valid, y_pred)
print(cr)